### IMPORTAR LIBREARÍAS

In [48]:
!pip install matplotlib

     ---------------------------------------- 7.2/7.2 MB 7.2 MB/s eta 0:00:00
     ------------------------------------- 965.4/965.4 kB 10.2 MB/s eta 0:00:00
     ---------------------------------------- 54.9/54.9 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 8.5 MB/s eta 0:00:00


In [97]:
import pandas as pd
import pymysql
import random 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

random.seed(42)

### Generar Dataframe a partir de tablas

In [98]:
conn = pymysql.connect(host='localhost', user='tfm_user', password='tfmdatabase1234', db='tfm_database')

inmueble_query = "SELECT * FROM inmueble"
anuncio_query = "SELECT * FROM anuncio"

df_inmueble = pd.read_sql(inmueble_query, conn)
df_anuncio = pd.read_sql(anuncio_query, conn)

inmueble_columns = ["id_inmueble", "exterior", "num_habitaciones", "superficie", "ascensor", "parking"]
anuncio_columns = ["id_anuncio", "id_inmueble", "precio"]

df_inmueble = df_inmueble[inmueble_columns]
df_anuncio = df_anuncio[anuncio_columns]
df = df_anuncio.merge(df_inmueble, how="left", on="id_inmueble")

### Transformar el Dataframe para ser entrenado

In [99]:
df_machine = pd.get_dummies(df)
x_column = list(set(df_machine.columns) - set(["precio", "id_anuncio", "id_inmueble"]))
y_column = ["precio"]
X = df_machine[x_column]
y = df_machine[y_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
scaler = StandardScaler()
scaler2 = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.fit_transform(X_test)

In [100]:
model = RandomForestRegressor()
model.fit(X_train_scale, y_train)

c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor()

In [101]:
y_train_predict = model.predict(X_train_scale)
y_test_predict = model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, y_train_predict))
print("Test Accuracy: ", r2_score(y_test, y_test_predict))

print("MSE Train: ", mean_squared_error(y_train, y_train_predict))
print("MSE Test: ", mean_squared_error(y_test, y_test_predict))

Train Accuracy:  0.9186951321043859
Test Accuracy:  0.7511626691590716
MSE Train:  49951549904.0334
MSE Test:  145589758796.9421


ESCALAR VARIABLE TARGET

In [102]:
y_train_scale = scaler.fit_transform(y_train)
y_test_scale = scaler.fit_transform(y_test)

In [103]:
model = RandomForestRegressor()
model.fit(X_train_scale, y_train_scale)

c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor()

In [104]:
y_train_predict = model.predict(X_train_scale)
y_test_predict = model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, scaler.inverse_transform(y_train_predict.reshape(-1, 1)).reshape(-1)))
print("Test Accuracy: ", r2_score(y_test, scaler.inverse_transform(y_test_predict.reshape(-1, 1)).reshape(-1)))

print("MSE Train: ", mean_squared_error(y_train, scaler.inverse_transform(y_train_predict.reshape(-1, 1)).reshape(-1)))
print("MSE Test: ", mean_squared_error(y_test, scaler.inverse_transform(y_test_predict.reshape(-1, 1)).reshape(-1)))

Train Accuracy:  0.916938352127292
Test Accuracy:  0.7541619496773246
MSE Train:  51030868830.04009
MSE Test:  143834939591.39465


AJUSTE DE PARÁMETROS


In [105]:
from sklearn.model_selection import GridSearchCV

In [124]:
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 10, 30],
    'max_features':['log2', 'sqrt', 'auto']
}

In [125]:
model_gridcv = GridSearchCV(model, params, cv=3)
model_gridcv.fit(X_train_scale, y_train)
best_model = model_gridcv.best_estimator_

c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-

In [126]:
best_model.fit(X_train_scale, y_train)

c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(max_depth=10)

In [127]:
y_train_predict = best_model.predict(X_train_scale)
y_test_predict = best_model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, y_train_predict))
print("Test Accuracy: ", r2_score(y_test, y_test_predict))

print("MSE Train: ", mean_squared_error(y_train, y_train_predict))
print("MSE Test: ", mean_squared_error(y_test, y_test_predict))

Train Accuracy:  0.8885019524268277
Test Accuracy:  0.7835318768755155
MSE Train:  68501437019.78816
MSE Test:  126651181020.2106


In [128]:
print(model_gridcv.best_params_)

{'max_depth': 10, 'max_features': 'auto', 'n_estimators': 100}


OTRO MODELO

In [129]:
from xgboost import XGBRegressor

In [130]:
xgb_model = XGBRegressor()

params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 10, 30],
    'learning_rate': [0.1, 0.01, 0.001]
}

xgb_model_gridcv = GridSearchCV(xgb_model, params, cv=3)
xgb_model_gridcv.fit(X_train_scale, y_train)
best_xgb_model = xgb_model_gridcv.best_estimator_

In [131]:
best_xgb_model.fit(X_train_scale, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [132]:
y_train_predict = best_xgb_model.predict(X_train_scale)
y_test_predict = best_xgb_model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, y_train_predict))
print("Test Accuracy: ", r2_score(y_test, y_test_predict))

print("MSE Train: ", mean_squared_error(y_train, y_train_predict))
print("MSE Test: ", mean_squared_error(y_test, y_test_predict))

Train Accuracy:  0.9154817965178385
Test Accuracy:  0.7538961917210714
MSE Train:  51925737883.96971
MSE Test:  143990429270.6101


In [133]:
print(xgb_model_gridcv.best_params_)

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 50}
